In [27]:
import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime




rm = visa.ResourceManager()
rm.list_resources()

('TCPIP0::AT::inst0::INSTR',
 'ASRL3::INSTR',
 'ASRL10::INSTR',
 'GPIB0::3::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::8::INSTR',
 'GPIB0::14::INSTR',
 'GPIB0::30::INSTR')

In [28]:
#hall measurement connection, chan 1 rxy, chan 2 rxx

pulser = rm.open_resource('GPIB0::7::INSTR')


In [29]:
def applyPulse(pulser, channel=1, width=0.5, voltage=10, delay=0,t0_period=4):
    for i in range(1, 5):
        if i == channel:
            pulser.write(":PULSE{}:STATE 1".format(i))
        else:
            pulser.write(":PULSE{}:STATE 0".format(i))
    
    pulser.write(":PULSE0:period {}".format(t0_period))
    pulser.write(":PULSE0:mode single")
    
    pulser.write(":PULSE{}:delay {}".format(channel, delay))
    pulser.write(":PULSE{}:width {}".format(channel, width))
    pulser.write(":PULSE{}:polarity Normal".format(channel))
    pulser.write(":PULSE{}:output:amplitude {}".format(channel, voltage))
    
    pulser.write(":PULSE0:STATE 1")
    
    
    
    

In [47]:
applyPulse(pulser=pulser, channel=2, width=0.001, voltage=40, delay=0, t0_period=0.001)

In [30]:
#switch

dc_2400_id = 30
nanovolt_id = 8


dc = Keithley2400("GPIB::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [31]:
def init_communication(amp=0.2):
    print("check ac_1: amp {}V, duration 5s".format(amp))
    
    applyPulse(pulser=pulser, channel=1, width=5, voltage=amp, delay=0, t0_period=6)
    time.sleep(10)
    
    print("check ac_2: amp {}V, duration 5s".format(amp))

    applyPulse(pulser=pulser, channel=2, width=5, voltage=amp, delay=0, t0_period=6)
    time.sleep(15)
    """
    print("check ac_mid: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_mid, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)"""
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = amp            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")

In [64]:
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    
    print("start sleep")
    time.sleep(7)
    print("end sleep")
    
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
   
    v_down = []
    

    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v

In [65]:
import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise

In [66]:
def do_measurement_three_pulses(sample_id='', ac_voltage=2, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_voltage * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                for t0 in range(10, 0, -3):
                    print("{} seconds before applying pulse".format(t0))
                    time.sleep(1)
                    
                if current == 'ac_1':
                    
                    applyPulse(pulser=pulser, channel=1, width=duration, voltage=ac_voltage, delay=0, t0_period=duration*3)
                
               
                elif current == 'ac_2':
                    
                    applyPulse(pulser=pulser, channel=2, width=duration, voltage=ac_voltage, delay=0, t0_period=duration*3)
                
                else:
                    print("no ac " + current)
                    raise
                
                time.sleep(2)
                print("pulse applied")
                for t0 in range(10, 0, -3):
                    time.sleep(1)
                    print("{} seconds before measuring resistance".format(t0))
                
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)


In [67]:
do_task = True
check_conn = True
anneal_at_high_T = False

In [36]:
init_communication(amp=0.2)

check ac_1: amp 0.2V, duration 5s
check ac_2: amp 0.2V, duration 5s
enable dc current now, current:<pymeasure.instruments.keithley.keithley2400.Keithley2400 object at 0x000001B5998305B0>
check nano volt:
-9.48088525E-06

-9.48088525E-06

-9.16245055E-06

-9.16245055E-06

-8.84321459E-06

-8.84321459E-06

-8.52377831E-06

-8.52377831E-06

-8.52551438E-06

-8.52551438E-06

dc current disabled


In [71]:
#three leg switch, up mid down mid up
#set_Field(0)
#time.sleep(60)

if not do_task:
    raise
#t_array = [100, 200, 250, 270, 280, 290, 300, 310, 320,330, 340, 350]
#t_array = [100, 150, 200, 225, 250, 275, 300, 325]
t_array = [None]
#i = [15*1e-3]
#     14*1e-3, 16*1e-3,16*1e-3, 14*1e-3, 12*1e-3,10*1e-3, 8*1e-3, 6*1e-3]
#i = [15*1e-3]#[::-1]


vs = [40]

#i = i + i[::-1]

#set_T(300)
#time.sleep(15*60)
#i = [10*1e-3]
#set_Field(30000, True)

pulse_durations = [0.001]
fields = [0]
wait_after_T = 5 # in minute
if check_conn:
    do_measurement_three_pulses(sample_id='100nsD664_1_2nmPt_1nmNiO30nm_Fe2O3__8leg_0dsegree_0T_switch_10sec_3sec__345Kanneal'+ "G",
                   ac_voltage=40, dc_current=10e-5, duration= 0.0000001, 
                   round=2, point_per_round=4, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])
    raise

if anneal_at_high_T:
    set_T(350)
    do_measurement_three_pulses(sample_id='D664_1_2nmPt_1nmNiO30nm_Fe2O3__8leg_0dsegree_0T_switch_10sec_3sec__345Kanneal'+ "G",
               ac_voltage=30, dc_current=10e-5, duration=0.001, 
               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    if not t is None:
        set_T(t)
        time.sleep(wait_after_T*60)
    
    for v in vs:
        for field in fields:
            for pulse_d in pulse_durations:
                print("current pulse:" + str(v))
                print("set field to:" + str(field))
                #set_Field(field)
                print(datetime.datetime.now())
                do_measurement_three_pulses(sample_id='D664_1_2nmPt_1nmNiO30nm_Fe2O3__8leg_0dsegree_0T_switch_10sec_3sec_345Kanneal'+ str(t) +"K" + str(field) + "G",
                               ac_voltage=v, dc_current=10e-5, duration=pulse_d, 
                               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

do_task = False

start sleep
end sleep
round: 1
current:ac_1
10 seconds before applying pulse
7 seconds before applying pulse
4 seconds before applying pulse
1 seconds before applying pulse
pulse applied
10 seconds before measuring resistance
7 seconds before measuring resistance
4 seconds before measuring resistance
1 seconds before measuring resistance
start sleep
end sleep
-0.0005699533376428572
10 seconds before applying pulse
7 seconds before applying pulse
4 seconds before applying pulse
1 seconds before applying pulse
pulse applied
10 seconds before measuring resistance
7 seconds before measuring resistance
4 seconds before measuring resistance
1 seconds before measuring resistance
start sleep
end sleep
-0.0005697076938928573
10 seconds before applying pulse
7 seconds before applying pulse
4 seconds before applying pulse
1 seconds before applying pulse
pulse applied
10 seconds before measuring resistance
7 seconds before measuring resistance
4 seconds before measuring resistance
1 seconds before

KeyboardInterrupt: 